<a href="https://colab.research.google.com/github/mohamed2507/Rain-prediction-in-Australia/blob/master/Australia_rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohamedbenmaiza","key":"37ad3fb646a73e153fbf9b92783cbae4"}'}

In [3]:
! mkdir ~/.kaggle 

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
 ! chmod 777 ~/.kaggle/kaggle.json

In [6]:
 ! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              235KB  2021-10-01 17:39:01          13465  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           8084  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           3569  
fatiimaezzahra/famous-iconic-women                          Famous Iconic Women                               838MB  2021-02-28 14:56:00           1538  
dhruvildave/wikibooks-dataset                               Wikibooks Datase

In [7]:
!kaggle datasets download -d jsphyg/weather-dataset-rattle-package

  0% 0.00/3.83M [00:00<?, ?B/s]
100% 3.83M/3.83M [00:00<00:00, 63.5MB/s]


In [8]:
! unzip weather-dataset-rattle-package.zip -d /content/



Archive:  weather-dataset-rattle-package.zip
  inflating: /content/weatherAUS.csv  


In [9]:
import pandas as pd

In [10]:
data =pd.read_csv("/content/weatherAUS.csv")

In [11]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [12]:
data.shape

(145460, 23)

In [20]:
data.isna().any()

Date             False
Location         False
MinTemp          False
MaxTemp          False
Rainfall         False
Evaporation      False
Sunshine         False
WindGustDir      False
WindGustSpeed    False
WindDir9am       False
WindDir3pm       False
WindSpeed9am     False
WindSpeed3pm     False
Humidity9am      False
Humidity3pm      False
Pressure9am      False
Pressure3pm      False
Cloud9am         False
Cloud3pm         False
Temp9am          False
Temp3pm          False
RainToday        False
RainTomorrow     False
dtype: bool

In [14]:
data['MinTemp']=data['MinTemp'].fillna(data['MinTemp'].mean())

In [15]:
for t in data.loc[:,data.isna().any()]:
  if(data[t].dtype=="float64"):
    data[t]=data[t].fillna(data[t].mean())



In [17]:
data.shape[0]-data.isna().any(axis=1).sum()

123710

In [18]:
data=data[data.isna().any(axis=1)==False]

In [19]:
data.loc[:,data.isna().any()]

""
0
1
2
3
4
...
145454
145455
145456
145457


In [21]:
data=data.drop(labels="Date",axis=1)

In [22]:
for t in data:
  if(data[t].dtype=="object"):
    print(t)
data=pd.get_dummies(data,columns=["Location","WindGustDir","WindDir9am","WindDir3pm","RainToday"])


Location
WindGustDir
WindDir9am
WindDir3pm
RainToday
RainTomorrow


In [23]:
data.loc[data["RainTomorrow"]=="No","RainTomorrow"]=0
data.loc[data["RainTomorrow"]=="Yes","RainTomorrow"]=1
data["RainTomorrow"].value_counts()

0    96318
1    27392
Name: RainTomorrow, dtype: int64

In [24]:
data.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,Location_Adelaide,Location_Albury,Location_AliceSprings,Location_BadgerysCreek,Location_Ballarat,Location_Bendigo,Location_Brisbane,Location_Cairns,Location_Canberra,Location_Cobar,Location_CoffsHarbour,Location_Dartmoor,Location_Darwin,Location_GoldCoast,Location_Hobart,Location_Katherine,Location_Launceston,Location_Melbourne,Location_MelbourneAirport,Location_Mildura,Location_Moree,Location_MountGambier,Location_MountGinini,...,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_E,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
0,13.4,22.9,0.6,5.468232,7.611178,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,7.4,25.1,0.0,5.468232,7.611178,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,12.9,25.7,0.0,5.468232,7.611178,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,9.2,28.0,0.0,5.468232,7.611178,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,17.5,32.3,1.0,5.468232,7.611178,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [25]:
print(data.isna().any().sum())
data=data[data.isna().any(axis=1)==False]
data.isna().any().sum()

0


0

In [28]:
d0=data[data['RainTomorrow']==0].sample(50000)
d1=data[data['RainTomorrow']==1].sample(50000,replace=True)
data=pd.concat([d0,d1])

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(data.loc[:,data.columns!="RainTomorrow"],data.loc[:,"RainTomorrow"],test_size=0.2,shuffle=True)

In [45]:
X_test.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Location_Adelaide,Location_Albury,Location_AliceSprings,Location_BadgerysCreek,Location_Ballarat,Location_Bendigo,Location_Brisbane,Location_Cairns,Location_Canberra,Location_Cobar,Location_CoffsHarbour,Location_Dartmoor,Location_Darwin,Location_GoldCoast,Location_Hobart,Location_Katherine,Location_Launceston,Location_Melbourne,Location_MelbourneAirport,Location_Mildura,Location_Moree,Location_MountGambier,Location_MountGinini,Location_Nhil,...,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_E,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_E,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
83827,18.4,29.4,23.8,5.468232,7.611178,72.0,35.0,46.0,94.0,68.0,993.3,997.0,4.447461,4.50993,20.9,27.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
112675,17.5,25.6,0.2,5.468232,7.611178,46.0,20.0,28.0,70.0,62.0,1009.3,1008.5,4.447461,4.50993,21.5,22.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
7238,5.0,18.4,0.0,3.600000,7.611178,35.0,9.0,19.0,70.0,39.0,1023.8,1020.1,0.000000,3.00000,11.3,17.2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
84580,21.5,32.5,0.0,9.000000,11.600000,28.0,7.0,13.0,59.0,47.0,1015.0,1012.2,5.000000,3.00000,28.9,31.9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
28100,6.0,17.0,0.0,6.600000,7.611178,35.0,13.0,19.0,73.0,59.0,1038.7,1036.8,4.447461,4.50993,13.7,17.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [46]:
X_train=X_train.to_numpy().astype('float32')
Y_train=Y_train.to_numpy().astype('float32')
X_test=X_test.to_numpy().astype('float32')
Y_test=Y_test.to_numpy().astype('float32')

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import regularizers
import tensorflow as tf

In [48]:
model=Sequential([
               Dense(16,input_shape=(113,),activation="relu"),
               Dense(32,activation="relu"),
               Dense(64,activation="relu"),
               Dense(128,activation="relu"),
               Dense(256,activation="relu"),
               Dense(1,activation="sigmoid"),   
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 16)                1824      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_20 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_21 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_22 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 257       
Total params: 46,081
Trainable params: 46,081
Non-trainable params: 0
__________________________________________________

In [49]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [50]:
callback=tf.keras.callbacks.EarlyStopping(patience=3)

In [51]:
model.fit(X_train,Y_train,epochs=10,batch_size=32,validation_data=(X_test,Y_test),callbacks=[callback])

Epoch 1/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4945 - accuracy: 0.7609 - val_loss: 0.4704 - val_accuracy: 0.7745
Epoch 2/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4593 - accuracy: 0.7834 - val_loss: 0.4870 - val_accuracy: 0.7623
Epoch 3/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4571 - accuracy: 0.7849 - val_loss: 0.4510 - val_accuracy: 0.7880
Epoch 4/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4549 - accuracy: 0.7869 - val_loss: 0.4658 - val_accuracy: 0.7778
Epoch 5/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4524 - accuracy: 0.7885 - val_loss: 0.4495 - val_accuracy: 0.7898
Epoch 6/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4507 - accuracy: 0.7884 - val_loss: 0.4492 - val_accuracy: 0.7896
Epoch 7/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.4495 - accuracy: 0.7897 - val_loss: 0.4631 - val_accuracy:

In [52]:
model.evaluate(X_test,Y_test)

625/625 [==============================] - 1s 1ms/step - loss: 0.4559 - accuracy: 0.7811


[0.4559234380722046, 0.7810999751091003]

In [53]:
prediction=model.predict(X_test)
prediction[prediction>0.5]=1
prediction[prediction<0.5]=0

In [54]:
from sklearn.metrics import confusion_matrix
c=confusion_matrix(Y_test,prediction)


In [56]:
precision =c[0,0]/(c[0,0]+c[0,1])
recall =c[0,0]/(c[0,0]+c[1,0])
f1_score=2*(precision*recall)/(precision+recall)
print('f1_score: '+str(f1_score ))
print("precision: ",precision)
print("recall: ",recall)

f1_score: 0.7934126085315213
precision:  0.8422159887798036
recall:  0.7499553969669938
